# Report

In this notebook we report progress on the project of house price prediction.

In [1]:
import os
from functools import partial

In [2]:
import joblib
import pandas as pd

In [3]:
import data
import metrics

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
def evaluate_model(*, model, metric, X_train, y_train, X_test, y_test):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    train_error = metric(y_train, train_predictions)
    test_error = metric(y_test, test_predictions)
    return {
        "train_predictions": train_predictions,
        "test_predictions": test_predictions,
        "train_error": train_error,
        "test_error": test_error
    }

def print_report(*, model, evaluation):
    print(f"Model used:\n\t{reg}")
    print(f"Error:\n\ttrain set {evaluation['train_error']}\n\ttest error: {evaluation['test_error']}")

In [6]:
models_dir = "models"
dataset_path = "./data/train_regression.csv"
dataset = data.get_dataset(
    partial(pd.read_csv, filepath_or_buffer=dataset_path),
    splits=("train", "test")
)

**If you need to visualize anything from your training data, do it here**

## Baseline

Before doing any complex Machine Learning model, let's try to solve the problem by having an initial educated guess. 

In [11]:
model_path = os.path.join("models", "Baseline", "model.joblib")
reg = joblib.load(model_path)
evaluation = evaluate_model(
    model=reg,
    metric=metrics.custom_error,
    X_train=dataset["train"][0],
    y_train=dataset["train"][1],
    X_test=dataset["test"][0],
    y_test=dataset["test"][1]
)
print_report(model=reg, evaluation=evaluation)

Model used:
	Pipeline(steps=[('average-charges-extractor',
                 AverageChargesPerRegionExtractor()),
                ('average-charges-regressor',
                 AverageChargesPerRegionRegressor())])
Error:
	train set 7763.483136076173
	test error: 8214.391166112087


## Linear Regression Model 

We want to try easy things first, so know lets see how a linear regression model does.

In [13]:
model_path = os.path.join("models", "2021-06-15 00-42", "model.joblib")
reg = joblib.load(model_path)
evaluation = evaluate_model(
    model=reg,
    metric=metrics.custom_error,
    X_train=dataset["train"][0],
    y_train=dataset["train"][1],
    X_test=dataset["test"][0],
    y_test=dataset["test"][1]
)
print_report(model=reg, evaluation=evaluation)

Model used:
	Pipeline(steps=[('average-charges-extractor',
                 AverageChargesPerRegionExtractor()),
                ('categorical-encoder',
                 CategoricalEncoder(force_dense_array=True, one_hot=True)),
                ('standard-scaler', StandardScaler()),
                ('linear-regressor', LinearRegression())])
Error:
	train set 3885.0639260784033
	test error: 3741.6139696012647


**Error Analysis**

What can you learn about the errors your model is making? Try this:

* Discretize the errors your model is making by some categorical variables.
* Sort and discretize the errors your model is making and see what the features have in common in those cases. 

## Linear regression with Feature Engineering

Probably the previous model is not good enough, let's see how is the performance of a model using some produced features.

Techniques:
1. Feature Cross
2. Discretizer
3. Add average per neighborhood.


In [36]:
model_path = os.path.join("models", "2021-06-15 01-28", "model.joblib")
reg = joblib.load(model_path)
evaluation = evaluate_model(
    model=reg,
    metric=metrics.custom_error,
    X_train=dataset["train"][0],
    y_train=dataset["train"][1],
    X_test=dataset["test"][0],
    y_test=dataset["test"][1]
)
print_report(model=reg, evaluation=evaluation)

Model used:
	Pipeline(steps=[('average-charges-extractor',
                 AverageChargesPerRegionExtractor()),
                ('discretizer',
                 Discretizer(bins_per_column={'bmi': 4}, strategy='quantile')),
                ('categorical-encoder',
                 CategoricalEncoder(force_dense_array=True, one_hot=True)),
                ('standard-scaler', StandardScaler()),
                ('linear-regressor', LinearRegression())])
Error:
	train set 4045.0999620912107
	test error: 3983.1166212763574


**Error Analysis**

What can you learn about the errors your model is making? Try this:

* Discretize the errors your model is making by some categorical variables.
* Sort or discretize the errors your model is making and see what the features have in common in those cases. 

### Carreta asociada